# 문서 검색에서의 문맥 기반 압축 (Contextual Compression in Document Retrieval)

## 개요

이 코드는 LangChain과 OpenAI의 언어 모델을 사용하여 문서 검색 시스템에서 문맥 기반 압축을 구현하는 예제입니다. 이 기술은 쿼리의 문맥에 맞춰 문서에서 가장 관련성이 높은 부분만을 압축하고 추출함으로써 검색 정보의 연관성과 간결성을 개선하는 것을 목표로 합니다.

## 동기

기존 문서 검색 시스템은 전체 청크나 문서를 반환하는 경우가 많아 불필요한 정보가 포함될 수 있습니다. 문맥 기반 압축은 쿼리에 따라 관련성 높은 부분만을 추출하여 집중적이고 효율적인 정보 검색을 가능하게 합니다.

## 주요 구성 요소

1. PDF 문서에서 벡터 스토어 생성
2. 기본 검색기(retriever) 설정
3. LLM(언어 모델) 기반의 문맥 압축기
4. 문맥 압축 검색기
5. 압축된 검색기를 통합한 질문-응답 체인

## 방법론

### 문서 전처리 및 벡터 스토어 생성

1. PDF는 `encode_pdf`라는 사용자 정의 함수를 통해 벡터 스토어로 인코딩됩니다.

### 검색기 및 압축기 설정

1. 벡터 스토어를 기반으로 기본 검색기를 생성합니다.
2. OpenAI의 GPT-4 모델을 사용하여 LLM 기반의 문맥 압축기 (`LLMChainExtractor`)를 초기화합니다.

### 문맥 기반 압축 검색기

1. 기본 검색기와 압축기를 결합하여 `ContextualCompressionRetriever`를 생성합니다.
2. 이 검색기는 우선 기본 검색기를 사용해 문서를 가져오고, 이후 압축기를 적용하여 가장 관련성이 높은 정보를 추출합니다.

### 질문-응답 체인

1. 압축된 검색기를 통합한 `RetrievalQA` 체인을 생성합니다.
2. 이 체인은 압축되고 추출된 정보를 사용해 쿼리에 대한 답변을 생성합니다.

## 이 접근법의 장점

1. **향상된 관련성**: 쿼리에 맞는 가장 관련성이 높은 정보만 반환합니다.
2. **높은 효율성**: 관련 부분을 압축하여 LLM이 처리해야 할 텍스트의 양을 줄입니다.
3. **문맥 이해도 향상**: LLM 기반 압축기가 쿼리의 문맥을 이해하여 정보 추출을 수행합니다.
4. **유연성**: 다양한 문서 유형과 쿼리에 쉽게 적응할 수 있습니다.

## 결론

문서 검색에서 문맥 기반 압축은 정보 검색 시스템의 품질과 효율성을 향상시키는 강력한 방법입니다. 관련 정보를 지능적으로 압축하고 추출하여 쿼리에 더욱 집중적이고 문맥에 맞는 응답을 제공합니다. 이 접근법은 대규모 문서 집합에서의 효율적이고 정확한 정보 검색이 필요한 다양한 분야에서 유용하게 적용될 수 있습니다.


<div style="text-align: center;">

<img src="../images/contextual_compression.svg" alt="contextual compression" style="width:70%; height:auto;">
</div>

### Import libraries

In [2]:
import os
import sys
from dotenv import load_dotenv
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.retrievers import ContextualCompressionRetriever
from langchain.chains import RetrievalQA


sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..'))) # Add the parent directory to the path sicnce we work with notebooks
from helper_functions import *
from evaluation.evalute_rag import *

# Load environment variables from a .env file
load_dotenv()

# Set the OpenAI API key environment variable
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

### Define document's path

In [3]:
path = "../data/Understanding_Climate_Change.pdf"

### Create a vector store

In [4]:
vector_store = encode_pdf(path)

### Create a retriever + contexual compressor + combine them 

In [5]:
# Create a retriever
retriever = vector_store.as_retriever()


#Create a contextual compressor
llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini", max_tokens=4000)
compressor = LLMChainExtractor.from_llm(llm)

#Combine the retriever with the compressor
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=retriever
)

# Create a QA chain with the compressed retriever
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=compression_retriever,
    return_source_documents=True
)

### Example usage

In [6]:
query = "What is the main topic of the document?"
result = qa_chain.invoke({"query": query})
print(result["result"])
print("Source documents:", result["source_documents"])

The main topic of the document is climate change, focusing on its impacts on human health, the importance of international collaboration and policy development, and the ethical dimensions of climate justice. It discusses various aspects such as health impacts, global agreements like the Paris Agreement, national climate policies, and the need for sustainable and equitable solutions for future generations.
Source documents: [Document(metadata={'source': '../data/Understanding_Climate_Change.pdf', 'page': 14}, page_content='Chapter 10: Climate Change and Human Health  \nHealth Impacts  \nHeat -Related Illnesses  \nRising temperatures and more frequent heatwaves increase the risk of heat -related illnesses, such as heat exhaustion and heatstroke. Vulnerable populations, including the elderly, children, and outdoor workers, are particularly at risk. Heat mitigation str ategies, such as cooling centers and public health campaigns, are essential.  \nVector -Borne Diseases  \nClimate change a

## 예시

In [8]:
# 기본 검색기로 검색된 문서에서 압축기만을 직접 사용하는 코드 예제

# 먼저, 쿼리를 통해 기본 검색기로부터 문서 검색
query = "What is the main topic of the document?"
retrieved_docs = retriever.get_relevant_documents(query)

# 검색된 문서 목록 출력
print("Retrieved Documents:")
for i, doc in enumerate(retrieved_docs):
    print(f"Document {i + 1}:\n", doc.page_content)
    print("=" * 80)

/var/folders/gl/6qlsk6x94klgs54qzsg9q1jm0000gn/T/ipykernel_3302/1143730304.py:5: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(query)


Retrieved Documents:
Document 1:
 goals. Policies should promote synergies between biodiversity conservation and climate 
action.  
Chapter 10: Climate Change and Human Health  
Health Impacts  
Heat -Related Illnesses  
Rising temperatures and more frequent heatwaves increase the risk of heat -related illnesses, 
such as heat exhaustion and heatstroke. Vulnerable populations, including the elderly, 
children, and outdoor workers, are particularly at risk. Heat mitigation str ategies, such as 
cooling centers and public health campaigns, are essential.  
Vector -Borne Diseases  
Climate change affects the distribution and prevalence of vector -borne diseases, such as 
malaria, dengue fever, and Lyme disease. Warmer temperatures and changing precipitation 
patterns can expand the habitats of disease -carrying insects, increasing the ri sk of outbreaks. 
Integrated pest management and disease surveillance are critical for prevention and control.  
Respiratory and Cardiovascular Diseases


In [10]:
# 압축기 적용: 검색된 문서들에서 가장 관련성 높은 부분만 추출
compressed_docs = compressor.compress_documents(
    documents=retrieved_docs,
    query=query
)

# 압축된 결과 출력
print("\nCompressed Documents:")
for i, compressed_doc in enumerate(compressed_docs):
    print(f"Compressed Document {i + 1}:\n", compressed_doc.page_content)  # "page_content"에 압축된 텍스트가 담김
    print("=" * 80)



Compressed Documents:
Compressed Document 1:
 Chapter 10: Climate Change and Human Health  
Health Impacts  
Heat -Related Illnesses  
Rising temperatures and more frequent heatwaves increase the risk of heat -related illnesses, such as heat exhaustion and heatstroke. Vulnerable populations, including the elderly, children, and outdoor workers, are particularly at risk. Heat mitigation str ategies, such as cooling centers and public health campaigns, are essential.  
Vector -Borne Diseases  
Climate change affects the distribution and prevalence of vector -borne diseases, such as malaria, dengue fever, and Lyme disease. Warmer temperatures and changing precipitation patterns can expand the habitats of disease -carrying insects, increasing the ri sk of outbreaks.  
Respiratory and Cardiovascular Diseases
Compressed Document 2:
 Chapter 6: Global and Local Climate Action  
International Collaboration  
United Nations Framework Convention on Climate Change (UNFCCC)  
The UNFCCC is an int

## 내 생각 
- retriever로 쿼리와 관련된 문서를 1차적으로 검색한 후
- 검색한 문서를 압축기를 통해 2차적으로 쿼리와 관련된 내용만 압축하는 방법 